In [1]:
# import carla
# import math
# import random
# import time
# import numpy as np
# import cv2
# import subprocess
# import open3d as o3d
# import matplotlib
# import torch
# import sys
# import os

# # Append ByteTrack path (adjust as needed)
# sys.path.append("C:/Users/okeiy/OneDrive - University of Salford/Documents/Sch Notes/Dissertation/Codes/Dataset/ByteTrack-ONNX-Sample-main")
# from byte_tracker.tracker.byte_tracker import BYTETracker

# # Create a new Carla client and retrieve world, blueprint library, and spawn points
# client = carla.Client('localhost', 2000)
# client.set_timeout(10.0)
# world = client.get_world()
# bp_lib = world.get_blueprint_library()
# spawn_points = world.get_map().get_spawn_points()

# # Spawn a vehicle (Dodge Charger) and disable autopilot for manual control
# vehicle_bp = bp_lib.find('vehicle.dodge.charger')
# vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
# if vehicle is None:
#     raise Exception("Vehicle could not be spawned. Check spawn points or simulation state.")
# vehicle.set_autopilot(False)

# # Adjust spectator to follow the vehicle
# spectator = world.get_spectator()
# transform = carla.Transform(
#     vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)),
#     vehicle.get_transform().rotation)
# spectator.set_transform(transform)

# # Start background traffic processes (optional)
# traffic_light_process = subprocess.Popen(['python', 'traffic_light_controller.py'])
# traffic_process = subprocess.Popen(['python', 'generate_traffic.py', '--number-of-vehicles', '10', '--number-of-walkers', '230'])
# time.sleep(5)

# # (Although a traffic manager exists, we override control manually)
# traffic_manager = client.get_trafficmanager()

# # Define color maps for overlays (used for additional sensors in the full code)
# VIRIDIS = np.array(matplotlib.colormaps.get_cmap('plasma').colors)
# VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
# COOL_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
# COOL = np.array(matplotlib.colormaps.get_cmap('winter')(COOL_RANGE))
# COOL = COOL[:, :3]  # Remove alpha channel

# # Set up the RGB camera sensor
# camera_init_trans = carla.Transform(carla.Location(x=0.0, y=0.0, z=1.6), carla.Rotation(pitch=0.0))
# def set_camera_attributes(camera_bp):
#     camera_bp.set_attribute('image_size_x', '800')
#     camera_bp.set_attribute('image_size_y', '600')
#     camera_bp.set_attribute('fov', '90')

# rgb_camera_bp = bp_lib.find('sensor.camera.rgb')
# set_camera_attributes(rgb_camera_bp)
# rgb_camera = world.spawn_actor(rgb_camera_bp, camera_init_trans, attach_to=vehicle)
# print("RGB camera sensor attached.")

# # Create an OpenCV window for visualization (detection overlays will be shown here)
# cv2.namedWindow("RGB Camera", cv2.WINDOW_NORMAL)
# cv2.resizeWindow("RGB Camera", 800, 600)

# # Initialize sensor data storage
# image_w = 800
# image_h = 600
# sensor_data = {
#     'rgb_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
#     'rgb_image_raw': None  # This will be updated by the camera callback
# }
# def add_label(image, label, position=(10, 50), font_scale=0.8, color=(255, 255, 255)):
#     labeled_image = image.copy()
#     cv2.putText(labeled_image, label, position, cv2.FONT_HERSHEY_SIMPLEX,
#                 font_scale, color, 2, cv2.LINE_AA)
#     return labeled_image

# def rgb_callback(image, data_dict):
#     # Convert raw image data to a (height x width x 4) array
#     img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
#     data_dict['rgb_image_raw'] = img.copy()  # Store the raw image
#     # For display, add a static label overlay (detection overlays will be added later)
#     data_dict['rgb_image'] = add_label(img, "RGB Camera")

# # Start listening to the RGB camera sensor
# rgb_camera.listen(lambda image: rgb_callback(image, sensor_data))
# # Load YOLO model (adjust model_path as needed)
# model_path = r"C:\Users\okeiy\OneDrive - University of Salford\Documents\Sch Notes\Dissertation\Codes\Dataset\Yolo\bdd100k\my_yolov11_model.pt"
# model = torch.load(model_path, map_location=torch.device('cuda'))
# model.model.eval()  # Set YOLO model to evaluation mode

# # Set up ByteTrack tracker
# class TrackerArgs:
#     def __init__(self):
#         self.track_thresh = 0.5   # Detection score threshold for tracking
#         self.track_buffer = 30    # Frames to keep lost tracks
#         self.match_thresh = 0.8   # Matching threshold
#         self.mot20 = False

# args_tracker = TrackerArgs()
# tracker = BYTETracker(args_tracker, frame_rate=30)

# # Detection parameters and class names (assuming YOLO is trained on bdd100k)
# target_size = 640
# conf_thresh = 0.4
# class_names = ["bike", "bus", "car", "motor", "person", "rider",
#                "traffic light", "traffic sign", "train", "truck"]

# def letterbox(image, new_shape=(640, 640), color=(114, 114, 114)):
#     shape = image.shape[:2]  # (height, width)
#     ratio = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
#     new_unpad = (int(round(shape[1] * ratio)), int(round(shape[0] * ratio)))
#     dw = new_shape[1] - new_unpad[0]
#     dh = new_shape[0] - new_unpad[1]
#     dw /= 2
#     dh /= 2
#     resized = cv2.resize(image, new_unpad, interpolation=cv2.INTER_LINEAR)
#     top = int(round(dh - 0.1))
#     bottom = int(round(dh + 0.1))
#     left = int(round(dw - 0.1))
#     right = int(round(dw + 0.1))
#     padded = cv2.copyMakeBorder(resized, top, bottom, left, right,
#                                 cv2.BORDER_CONSTANT, value=color)
#     return padded, ratio, left, top

# def compute_iou(box1, box2):
#     x_left = max(box1[0], box2[0])
#     y_top = max(box1[1], box2[1])
#     x_right = min(box1[2], box2[2])
#     y_bottom = min(box1[3], box2[3])
#     if x_right < x_left or y_bottom < y_top:
#         return 0.0
#     inter_area = (x_right - box1[0]) * (y_bottom - box1[1])
#     box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
#     box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
#     iou = inter_area / float(box1_area + box2_area - inter_area + 1e-6)
#     return iou

# def compute_distance(bbox, image_shape):
#     # Updated heuristic: calibrate the constant for your camera setup.
#     box_width = bbox[2] - bbox[0]
#     distance = 300.0 / (box_width + 1)  # Adjust constant as needed
#     return distance

# # Define threshold distance (in meters) for braking
# PEDESTRIAN_DISTANCE_THRESHOLD = 6.0
# # Set up output video writer (optional)
# byte_output_dir = r"C:\Users\okeiy\Desktop\New folder"
# os.makedirs(byte_output_dir, exist_ok=True)
# byte_output_video_path = os.path.join(byte_output_dir, "Carla_Output_With_Control.mp4")
# fourcc = cv2.VideoWriter_fourcc(*"mp4v")
# byte_frame_width = image_w
# byte_frame_height = image_h
# byte_out = cv2.VideoWriter(byte_output_video_path, fourcc, 30, (byte_frame_width, byte_frame_height))

# prev_time = time.time()
# frame_count = 0
# overall_start_time = time.time()

# while True:
#     try:
#         # Retrieve the current raw RGB image
#         if sensor_data.get('rgb_image_raw') is not None:
#             duplicate_rgb = sensor_data['rgb_image_raw'].copy()
#         else:
#             duplicate_rgb = np.zeros((image_h, image_w, 4), dtype=np.uint8)
        
#         # Prepare frame for processing (drop alpha channel)
#         byte_frame = duplicate_rgb[:, :, :3].copy()
#         byte_frame = np.ascontiguousarray(byte_frame)
        
#         # Preprocess frame for YOLO input using letterbox resize
#         padded_frame, ratio, pad_w, pad_h = letterbox(byte_frame, (target_size, target_size))
#         img_rgb = cv2.cvtColor(padded_frame, cv2.COLOR_BGR2RGB)
#         img_normalized = img_rgb.astype(np.float32) / 255.0
#         input_tensor = torch.from_numpy(img_normalized).permute(2, 0, 1).unsqueeze(0)
        
#         # Run YOLO detection
#         with torch.no_grad():
#             results = model(input_tensor)
#         res = results[0]
#         boxes = res.boxes.xyxy.cpu().numpy()
#         scores = res.boxes.conf.cpu().numpy()
#         raw_labels = res.boxes.cls.cpu().numpy()
        
#         valid_inds = scores >= conf_thresh
#         boxes = boxes[valid_inds]
#         scores = scores[valid_inds]
#         labels = raw_labels[valid_inds]
        
#         # Convert YOLO detections to original image coordinates
#         detection_boxes_orig = []
#         for box in boxes:
#             x1_orig = (box[0] - pad_w) / ratio
#             y1_orig = (box[1] - pad_h) / ratio
#             x2_orig = (box[2] - pad_w) / ratio
#             y2_orig = (box[3] - pad_h) / ratio
#             detection_boxes_orig.append([x1_orig, y1_orig, x2_orig, y2_orig])
#         detection_boxes_orig = np.array(detection_boxes_orig)
        
#         # Prepare detections for ByteTrack update
#         detections = np.concatenate([boxes, scores.reshape(-1, 1)], axis=1)
#         tracks = tracker.update(detections, (target_size, target_size), (target_size, target_size))
        
#         # Flag to indicate if braking is needed
#         apply_brake = False
        
#         # Process each track from ByteTrack
#         for track in tracks:
#             x1, y1, x2, y2 = track.tlbr
#             # Convert track coordinates from padded to original image space
#             x1_orig = int((x1 - pad_w) / ratio)
#             y1_orig = int((y1 - pad_h) / ratio)
#             x2_orig = int((x2 - pad_w) / ratio)
#             y2_orig = int((y2 - pad_h) / ratio)
#             track_box_orig = [x1_orig, y1_orig, x2_orig, y2_orig]
            
#             # Match track with YOLO detections using IoU to obtain class label
#             best_iou = 0
#             best_label = -1
#             for i, det_box in enumerate(detection_boxes_orig):
#                 iou = compute_iou(track_box_orig, det_box)
#                 if iou > best_iou:
#                     best_iou = iou
#                     best_label = int(labels[i])
#             if best_iou < 0.3:
#                 best_label = -1
            
#             # Check if this track corresponds to a pedestrian ("person")
#             if best_label >= 0 and best_label < len(class_names) and class_names[best_label] == "person":
#                 # Draw bounding box and label (with track ID and estimated distance)
#                 cv2.rectangle(byte_frame, (x1_orig, y1_orig), (x2_orig, y2_orig), (0, 255, 0), 2)
#                 distance = compute_distance(track_box_orig, byte_frame.shape)
#                 label_text = f"ID:{track.track_id} {distance:.1f}m"
#                 cv2.putText(byte_frame, label_text, (x1_orig, y1_orig - 10),
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
#                 print(f"Pedestrian ID {track.track_id} at distance: {distance:.2f}m")
#                 # If any pedestrian is too close, trigger braking
#                 if distance < PEDESTRIAN_DISTANCE_THRESHOLD:
#                     apply_brake = True
        
#         # Apply vehicle control based on pedestrian proximity
#         if apply_brake:
#             control = carla.VehicleControl(throttle=0.0, brake=1.0)
#             print("Pedestrian too close! Applying brakes.")
#         else:
#             control = carla.VehicleControl(throttle=0.5, brake=0.0)
#             print("Path clear. Driving normally.")
#         vehicle.apply_control(control)
        
#         # Compute FPS and overlay on frame
#         current_time = time.time()
#         fps = 1.0 / (current_time - prev_time)
#         prev_time = current_time
#         cv2.putText(byte_frame, f"FPS: {fps:.2f}", (10, 30),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
#         # Display the processed frame in the "RGB Camera" window
#         cv2.imshow("RGB Camera", byte_frame)
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
        
#         # Write frame to output video (optional)
#         byte_out.write(byte_frame)
#         frame_count += 1
#         time.sleep(0.001)
        
#     except Exception as e:
#         print("Error:", e)
#         break

# overall_end_time = time.time()
# total_time = overall_end_time - overall_start_time
# avg_fps = frame_count / total_time
# print(f"Average FPS: {avg_fps:.2f}")
# # Cleanup
# cv2.destroyAllWindows()
# byte_out.release()
# vehicle.destroy()
# print("Simulation ended and sensors cleaned up.")

In [1]:
import carla
import math
import random
import time
import numpy as np
import cv2
import subprocess
import open3d as o3d
import matplotlib
import torch
import sys
import os

from tensorflow.keras.models import load_model  # Import Keras to load your PilotNet model
from tensorflow.keras.losses import MeanSquaredError  # Import MeanSquaredError for custom_objects

# Append ByteTrack path (adjust as needed)
sys.path.append("C:/Users/okeiy/OneDrive - University of Salford/Documents/Sch Notes/Dissertation/Codes/Dataset/ByteTrack-ONNX-Sample-main")
from byte_tracker.tracker.byte_tracker import BYTETracker

# Create a new Carla client and retrieve world, blueprint library, and spawn points
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

# Spawn a vehicle (Dodge Charger) and disable autopilot for manual control
vehicle_bp = bp_lib.find('vehicle.dodge.charger')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
if vehicle is None:
    raise Exception("Vehicle could not be spawned. Check spawn points or simulation state.")
vehicle.set_autopilot(False)

# Adjust spectator to follow the vehicle
spectator = world.get_spectator()
transform = carla.Transform(
    vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)),
    vehicle.get_transform().rotation)
spectator.set_transform(transform)

# Start background traffic processes (optional)
traffic_light_process = subprocess.Popen(['python', 'traffic_light_controller.py'])
traffic_process = subprocess.Popen(['python', 'generate_traffic.py', '--number-of-vehicles', '10', '--number-of-walkers', '230'])
time.sleep(5)

# (Although a traffic manager exists, we override control manually)
traffic_manager = client.get_trafficmanager()

# Define color maps for overlays (used for additional sensors in the full code)
VIRIDIS = np.array(matplotlib.colormaps.get_cmap('plasma').colors)
VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
COOL_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
COOL = np.array(matplotlib.colormaps.get_cmap('winter')(COOL_RANGE))
COOL = COOL[:, :3]  # Remove alpha channel

# Set up the RGB camera sensor
camera_init_trans = carla.Transform(carla.Location(x=0.0, y=0.0, z=1.6), carla.Rotation(pitch=0.0))
def set_camera_attributes(camera_bp):
    camera_bp.set_attribute('image_size_x', '800')
    camera_bp.set_attribute('image_size_y', '600')
    camera_bp.set_attribute('fov', '90')

rgb_camera_bp = bp_lib.find('sensor.camera.rgb')
set_camera_attributes(rgb_camera_bp)
rgb_camera = world.spawn_actor(rgb_camera_bp, camera_init_trans, attach_to=vehicle)
print("RGB camera sensor attached.")

# Create an OpenCV window for visualization (detection overlays will be shown here)
cv2.namedWindow("RGB Camera", cv2.WINDOW_NORMAL)
cv2.resizeWindow("RGB Camera", 800, 600)

# Initialize sensor data storage
image_w = 800
image_h = 600
sensor_data = {
    'rgb_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
    'rgb_image_raw': None  # This will be updated by the camera callback
}
def add_label(image, label, position=(10, 50), font_scale=0.8, color=(255, 255, 255)):
    labeled_image = image.copy()
    cv2.putText(labeled_image, label, position, cv2.FONT_HERSHEY_SIMPLEX,
                font_scale, color, 2, cv2.LINE_AA)
    return labeled_image

def rgb_callback(image, data_dict):
    # Convert raw image data to a (height x width x 4) array
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    data_dict['rgb_image_raw'] = img.copy()  # Store the raw image
    # For display, add a static label overlay (detection overlays will be added later)
    data_dict['rgb_image'] = add_label(img, "RGB Camera")

# Start listening to the RGB camera sensor
rgb_camera.listen(lambda image: rgb_callback(image, sensor_data))

# Load YOLO model (adjust model_path as needed)
model_path = r"C:\Users\okeiy\OneDrive - University of Salford\Documents\Sch Notes\Dissertation\Codes\Dataset\Yolo\bdd100k\my_yolov11_model.pt"
model = torch.load(model_path, map_location=torch.device('cuda'))
model.model.eval()  # Set YOLO model to evaluation mode

# Load the PilotNet model for driving control with custom_objects to resolve "mse"
pilotnet_model_path = "C:/Users/okeiy/OneDrive - University of Salford/Documents/Sch Notes/Dissertation/Codes/Carla_dataset/pilotnet_model.h5"
pilotnet_model = load_model(pilotnet_model_path, custom_objects={'mse': MeanSquaredError()})

# Set up ByteTrack tracker
class TrackerArgs:
    def __init__(self):
        self.track_thresh = 0.5   # Detection score threshold for tracking
        self.track_buffer = 30    # Frames to keep lost tracks
        self.match_thresh = 0.8   # Matching threshold
        self.mot20 = False

args_tracker = TrackerArgs()
tracker = BYTETracker(args_tracker, frame_rate=30)

# Detection parameters and class names (assuming YOLO is trained on bdd100k)
target_size = 640
conf_thresh = 0.4
class_names = ["bike", "bus", "car", "motor", "person", "rider",
               "traffic light", "traffic sign", "train", "truck"]

def letterbox(image, new_shape=(640, 640), color=(114, 114, 114)):
    shape = image.shape[:2]  # (height, width)
    ratio = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    new_unpad = (int(round(shape[1] * ratio)), int(round(shape[0] * ratio)))
    dw = new_shape[1] - new_unpad[0]
    dh = new_shape[0] - new_unpad[1]
    dw /= 2
    dh /= 2
    resized = cv2.resize(image, new_unpad, interpolation=cv2.INTER_LINEAR)
    top = int(round(dh - 0.1))
    bottom = int(round(dh + 0.1))
    left = int(round(dw - 0.1))
    right = int(round(dw + 0.1))
    padded = cv2.copyMakeBorder(resized, top, bottom, left, right,
                                cv2.BORDER_CONSTANT, value=color)
    return padded, ratio, left, top

def compute_iou(box1, box2):
    x_left = max(box1[0], box2[0])
    y_top = max(box1[1], box2[1])
    x_right = min(box1[2], box2[2])
    y_bottom = min(box1[3], box2[3])
    if x_right < box1[0] or y_bottom < box1[1]:
        return 0.0
    inter_area = (x_right - box1[0]) * (y_bottom - box1[1])
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    iou = inter_area / float(box1_area + box2_area - inter_area + 1e-6)
    return iou

def compute_distance(bbox, image_shape):
    # Updated heuristic: calibrate the constant for your camera setup.
    box_width = bbox[2] - bbox[0]
    distance = 300.0 / (box_width + 1)  # Adjust constant as needed
    return distance

# Define threshold distance (in meters) for braking
PEDESTRIAN_DISTANCE_THRESHOLD = 5.0

# Set up output video writer (optional)
byte_output_dir = r"C:\Users\okeiy\Desktop\New folder"
os.makedirs(byte_output_dir, exist_ok=True)
byte_output_video_path = os.path.join(byte_output_dir, "Carla_Output_With_Control.mp4")
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
byte_frame_width = image_w
byte_frame_height = image_h
byte_out = cv2.VideoWriter(byte_output_video_path, fourcc, 30, (byte_frame_width, byte_frame_height))

prev_time = time.time()
frame_count = 0
overall_start_time = time.time()

while True:
    try:
        # Retrieve the current raw RGB image
        if sensor_data.get('rgb_image_raw') is not None:
            duplicate_rgb = sensor_data['rgb_image_raw'].copy()
        else:
            duplicate_rgb = np.zeros((image_h, image_w, 4), dtype=np.uint8)
        
        # Prepare frame for processing (drop alpha channel)
        byte_frame = duplicate_rgb[:, :, :3].copy()
        byte_frame = np.ascontiguousarray(byte_frame)
        
        # Preprocess frame for YOLO input using letterbox resize
        padded_frame, ratio, pad_w, pad_h = letterbox(byte_frame, (target_size, target_size))
        img_rgb = cv2.cvtColor(padded_frame, cv2.COLOR_BGR2RGB)
        img_normalized = img_rgb.astype(np.float32) / 255.0
        input_tensor = torch.from_numpy(img_normalized).permute(2, 0, 1).unsqueeze(0)
        
        # Run YOLO detection
        with torch.no_grad():
            results = model(input_tensor)
        res = results[0]
        boxes = res.boxes.xyxy.cpu().numpy()
        scores = res.boxes.conf.cpu().numpy()
        raw_labels = res.boxes.cls.cpu().numpy()
        
        valid_inds = scores >= conf_thresh
        boxes = boxes[valid_inds]
        scores = scores[valid_inds]
        labels = raw_labels[valid_inds]
        
        # Convert YOLO detections to original image coordinates
        detection_boxes_orig = []
        for box in boxes:
            x1_orig = (box[0] - pad_w) / ratio
            y1_orig = (box[1] - pad_h) / ratio
            x2_orig = (box[2] - pad_w) / ratio
            y2_orig = (box[3] - pad_h) / ratio
            detection_boxes_orig.append([x1_orig, y1_orig, x2_orig, y2_orig])
        detection_boxes_orig = np.array(detection_boxes_orig)
        
        # Prepare detections for ByteTrack update
        detections = np.concatenate([boxes, scores.reshape(-1, 1)], axis=1)
        tracks = tracker.update(detections, (target_size, target_size), (target_size, target_size))
        
        # Flag to indicate if braking is needed
        apply_brake = False
        
        # Process each track from ByteTrack
        for track in tracks:
            x1, y1, x2, y2 = track.tlbr
            # Convert track coordinates from padded to original image space
            x1_orig = int((x1 - pad_w) / ratio)
            y1_orig = int((y1 - pad_h) / ratio)
            x2_orig = int((x2 - pad_w) / ratio)
            y2_orig = int((y2 - pad_h) / ratio)
            track_box_orig = [x1_orig, y1_orig, x2_orig, y2_orig]
            
            # Match track with YOLO detections using IoU to obtain class label
            best_iou = 0
            best_label = -1
            for i, det_box in enumerate(detection_boxes_orig):
                iou = compute_iou(track_box_orig, det_box)
                if iou > best_iou:
                    best_iou = iou
                    best_label = int(labels[i])
            if best_iou < 0.3:
                best_label = -1
            
            # Check if this track corresponds to a pedestrian ("person")
            if best_label >= 0 and best_label < len(class_names) and class_names[best_label] == "person":
                # Draw bounding box and label (with track ID and estimated distance)
                cv2.rectangle(byte_frame, (x1_orig, y1_orig), (x2_orig, y2_orig), (0, 255, 0), 2)
                distance = compute_distance(track_box_orig, byte_frame.shape)
                label_text = f"ID:{track.track_id} {distance:.1f}m"
                cv2.putText(byte_frame, label_text, (x1_orig, y1_orig - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                print(f"Pedestrian ID {track.track_id} at distance: {distance:.2f}m")
                # If any pedestrian is too close, trigger braking
                if distance < PEDESTRIAN_DISTANCE_THRESHOLD:
                    apply_brake = True
        
        # Apply vehicle control based on pedestrian proximity or PilotNet model prediction
        if apply_brake:
            control = carla.VehicleControl(throttle=0.0, brake=1.0)
            print("Pedestrian too close! Applying brakes.")
        else:
            # Preprocess image for PilotNet model (resize to expected input dimensions, e.g., 200x66)
            pilotnet_input = cv2.resize(byte_frame, (200, 66))
            pilotnet_input = pilotnet_input.astype(np.float32) / 255.0  # Normalize to [0, 1]
            pilotnet_input = np.expand_dims(pilotnet_input, axis=0)  # Add batch dimension
            
            # Predict control outputs from PilotNet (assumes model outputs [steering, throttle])
            pred = pilotnet_model.predict(pilotnet_input)
            predicted_steering = float(pred[0][0])
            predicted_throttle = float(pred[0][1])
            
            # Scale the throttle output to provide enough acceleration
            scaling_factor = 900.0  # Adjust this factor based on your observations
            predicted_throttle_scaled = min(predicted_throttle * scaling_factor, 1.0)
            
            control = carla.VehicleControl(throttle=predicted_throttle_scaled, steer=predicted_steering, brake=0.0)
            print(f"Driving using PilotNet model. Steering: {predicted_steering:.3f}, "
                  f"Throttle (raw): {predicted_throttle:.3f}, "
                  f"Throttle (scaled): {predicted_throttle_scaled:.3f}")
        
        vehicle.apply_control(control)
        
        # Compute FPS and overlay on frame
        current_time = time.time()
        fps = 1.0 / (current_time - prev_time)
        prev_time = current_time
        cv2.putText(byte_frame, f"FPS: {fps:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Display the processed frame in the "RGB Camera" window
        cv2.imshow("RGB Camera", byte_frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
        # Write frame to output video (optional)
        byte_out.write(byte_frame)
        frame_count += 1
        time.sleep(0.001)
        
    except Exception as e:
        print("Error:", e)
        break

overall_end_time = time.time()
total_time = overall_end_time - overall_start_time
avg_fps = frame_count / total_time
print(f"Average FPS: {avg_fps:.2f}")
# Cleanup
cv2.destroyAllWindows()
byte_out.release()
vehicle.destroy()
print("Simulation ended and sensors cleaned up.")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
RGB camera sensor attached.


C:\Users\okeiy\AppData\Local\Temp\ipykernel_13668\2760746019.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=torch.device('c


0: 640x640 1 person, 1 traffic light, 1 traffic sign, 1 truck, 17.0ms
Speed: 2.0ms preprocess, 17.0ms inference, 119.4ms postprocess per image at shape (1, 3, 640, 640)
Pedestrian ID 1 at distance: 15.00m
Pedestrian ID 2 at distance: 2.59m
Pedestrian too close! Applying brakes.

0: 640x640 2 persons, 2 traffic lights, 1 truck, 21.5ms
Speed: 1.0ms preprocess, 21.5ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)
Pedestrian ID 2 at distance: 2.61m
Pedestrian too close! Applying brakes.

0: 640x640 2 persons, 2 traffic lights, 1 truck, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step
Driving using PilotNet model. Steering: -0.015, Throttle (raw): -0.009, Throttle (scaled): -8.143

0: 640x640 3 persons, 2 traffic lights, 1 truck, 18.0ms
Speed: 1.0ms preprocess, 18.0ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)
Pedestrian ID 2 at distance: 2.61m
Pedestrian too